<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_7_A_3D_truss_problem_geometry%2C_mesh%2C_loads_and_boundary_nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# ============================================================
# 3D Truss FEM (Problem 7) - MATLAB -> Python (Google Colab)
# copy & run in 1 cell
# ============================================================

def solution(GDof, prescribedDof, K, F, u_prescribed=None):
    if u_prescribed is None:
        u_prescribed = np.zeros(len(prescribedDof), dtype=float)

    prescribed = np.array([d-1 for d in prescribedDof], dtype=int)  # 0-based
    all_dofs = np.arange(GDof, dtype=int)
    free = np.setdiff1d(all_dofs, prescribed)

    U = np.zeros(GDof, dtype=float)
    U[prescribed] = u_prescribed

    K_ff = K[np.ix_(free, free)]
    K_fp = K[np.ix_(free, prescribed)]
    F_f  = F[free]

    rhs = F_f - K_fp @ U[prescribed]
    U[free] = np.linalg.solve(K_ff, rhs)
    return U


def outputDisplacementsReactions(U, K, GDof, prescribedDof, F):
    R = K @ U - F
    print("=== DISPLACEMENTS (U) ===")
    for i in range(GDof):
        print(f"DOF {i+1}: U = {U[i]:.6e}")

    print("\n=== REACTIONS AT PRESCRIBED DOFs ===")
    for d in prescribedDof:
        print(f"DOF {d}: R = {R[d-1]:.6e}")


def formStiffness3Dtruss(GDof, numberElements, elementNodes, nodeCoordinates, E, A):
    K = np.zeros((GDof, GDof), dtype=float)

    for e in range(numberElements):
        n1, n2 = elementNodes[e, 0], elementNodes[e, 1]  # 1-based nodes

        x1, y1, z1 = nodeCoordinates[n1-1, :]
        x2, y2, z2 = nodeCoordinates[n2-1, :]

        dx = x2 - x1
        dy = y2 - y1
        dz = z2 - z1
        L = np.sqrt(dx*dx + dy*dy + dz*dz)

        l = dx / L
        m = dy / L
        n = dz / L

        # 3D truss local stiffness in global coordinates (6x6)
        k = (E * A[e] / L) * np.array([
            [ l*l,  l*m,  l*n, -l*l, -l*m, -l*n],
            [ l*m,  m*m,  m*n, -l*m, -m*m, -m*n],
            [ l*n,  m*n,  n*n, -l*n, -m*n, -n*n],
            [-l*l, -l*m, -l*n,  l*l,  l*m,  l*n],
            [-l*m, -m*m, -m*n,  l*m,  m*m,  m*n],
            [-l*n, -m*n, -n*n,  l*n,  m*n,  n*n],
        ], dtype=float)

        # DOF map for node i: [3i-2, 3i-1, 3i]  (MATLAB 1-based)
        edof = np.array([
            3*n1-2, 3*n1-1, 3*n1,
            3*n2-2, 3*n2-1, 3*n2
        ], dtype=int) - 1  # to 0-based

        K[np.ix_(edof, edof)] += k

    return K


def stresses3Dtruss(numberElements, elementNodes, nodeCoordinates, U, E):
    sigma = np.zeros(numberElements, dtype=float)

    for e in range(numberElements):
        n1, n2 = elementNodes[e, 0], elementNodes[e, 1]

        x1, y1, z1 = nodeCoordinates[n1-1, :]
        x2, y2, z2 = nodeCoordinates[n2-1, :]

        dx = x2 - x1
        dy = y2 - y1
        dz = z2 - z1
        L = np.sqrt(dx*dx + dy*dy + dz*dz)

        l = dx / L
        m = dy / L
        n = dz / L

        edof = np.array([
            3*n1-2, 3*n1-1, 3*n1,
            3*n2-2, 3*n2-1, 3*n2
        ], dtype=int) - 1

        u_e = U[edof]

        # axial strain ~ ( [-l -m -n l m n] * u_e ) / L
        # stress = E * strain
        sigma[e] = (E / L) * np.array([-l, -m, -n, l, m, n], dtype=float) @ u_e

    print("\nstresses")
    print(sigma)
    return sigma


# ============================================================
# Main (equivalent to problem7.m)
# ============================================================

E = 1.2e6
A = np.array([0.302, 0.729, 0.187], dtype=float)  # per element areas

nodeCoordinates = np.array([
    [72.0, 0.0,   0.0],
    [0.0,  36.0,  0.0],
    [0.0,  36.0, 72.0],
    [0.0,   0.0, -48.0],
], dtype=float)

elementNodes = np.array([
    [1, 2],
    [1, 3],
    [1, 4],
], dtype=int)

numberElements = elementNodes.shape[0]
numberNodes = nodeCoordinates.shape[0]

GDof = 3 * numberNodes
U = np.zeros(GDof, dtype=float)
F = np.zeros(GDof, dtype=float)

# Applied load (MATLAB: force(3)=-1000)  => DOF3 = uz of node1
F[2] = -1000.0

# Stiffness
K = formStiffness3Dtruss(GDof, numberElements, elementNodes, nodeCoordinates, E, A)

# Boundary conditions (MATLAB: prescribedDof=[2 4:12]')
prescribedDof = [2] + list(range(4, 13))

# Solve
U = solution(GDof, prescribedDof, K, F)

# Output
outputDisplacementsReactions(U, K, GDof, prescribedDof, F)

# Element stresses
sigma = stresses3Dtruss(numberElements, elementNodes, nodeCoordinates, U, E)


=== DISPLACEMENTS (U) ===
DOF 1: U = -7.111436e-02
DOF 2: U = 0.000000e+00
DOF 3: U = -2.662391e-01
DOF 4: U = 0.000000e+00
DOF 5: U = 0.000000e+00
DOF 6: U = 0.000000e+00
DOF 7: U = 0.000000e+00
DOF 8: U = 0.000000e+00
DOF 9: U = 0.000000e+00
DOF 10: U = 0.000000e+00
DOF 11: U = 0.000000e+00
DOF 12: U = 0.000000e+00

=== REACTIONS AT PRESCRIBED DOFs ===
DOF 2: R = -2.231632e+02
DOF 4: R = 2.561226e+02
DOF 5: R = -1.280613e+02
DOF 6: R = 0.000000e+00
DOF 7: R = -7.024491e+02
DOF 8: R = 3.512245e+02
DOF 9: R = 7.024491e+02
DOF 10: R = 4.463264e+02
DOF 11: R = 0.000000e+00
DOF 12: R = 2.975509e+02

stresses
[ -948.19142387  1445.36842298 -2868.5433006 ]
